<a href="https://colab.research.google.com/github/tprasadtp/jupyter-notebooks/blob/master/dsp/Center_Channle_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Center Channel Extraction


In [0]:
# Deps
! pip install librosa

In [0]:
# Importing Stuff
import math
import numpy as np
import IPython.display as ipd
import numpy as np
from scipy.io import wavfile 
import os
import sys
import warnings

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

try:
    from IPython.display import Audio
except:
    warnings.warn("Can't import Audio from IPython.display; "
                  "Wave.audio() will not work.")


# Upload the Files
Please change the filenames accordigly in the later sections.

In [0]:
from google.colab import files
# Uploading Files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving impulse.wav to impulse.wav
Saving spirit.wav to spirit.wav
User uploaded file "impulse.wav" with length 379256 bytes
User uploaded file "spirit.wav" with length 7440766 bytes


## Deleting Old Files
Did you upload files twice? That can happen!. Delete them using normal shell commands.
just add `!` before the command. `!ls` will give you list of files here.

In [4]:
# For your Info You are running on
print ("------ Running on -------")
!uname -a
!lsb_release -a

------ Running on -------
Linux b2118e76c9c4 4.14.65+ #1 SMP Sun Sep 9 02:18:33 PDT 2018 x86_64 x86_64 x86_64 GNU/Linux
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.1 LTS
Release:	18.04
Codename:	bionic


# Rename your file / edit code
Rename Your files or edit the code below.

- If you wish to keep original code, rename your impulse response file to `impulse.wav` and audio file to `spirit.wav`. 
- IT **SHOULD** be **STEREO**

# Read Files

In [5]:
# Read Audio File
# Ignore warnings of chunk data if any.

print("Reading Audio file.....")
samp_rate, audio = wavfile.read("spirit.wav")

Reading Audio file.....


In [6]:
audio_rows, audio_cols = np.shape(audio)
if audio_cols == 2:
    print ('Source is Stereo file.')
    audio_l = audio[:,0]
    audio_r = audio[:,1]
    # Lets set a Flag.
    STEREO = True
elif audio_cols == 1:
    print('Source is Mono')
    print("Mono is not supported.")
    sys.exit(1)
else:
    print('Multi channel Audio is not suppoted')
    sys.exit(1)

if audio_rows / samp_rate  < 0.5:
    print('Audio is Too short to play with')
    sys.exit(1)
else:
    print ("Audio is {0:f} seconds long.".format(audio_rows/samp_rate))

Source is Stereo file.
Audio is 21.090476 seconds long.


# Processing Channels

- Normalize the audio samples b/w 1 & -1
- fft both channels
- compute center channel
- compute coefficients
- transform back to time domain
- write channels seperataely

In [8]:
def normalize(ys, amp=1.0):
    """Normalizes a wave array so the maximum amplitude is +amp or -amp.

    ys: wave array
    amp: max amplitude (pos or neg) in result

    returns: wave array
    """
    high, low = abs(max(ys)), abs(min(ys))
    return amp * ys / max(high, low)


print("Normalizing Numpy Arrays....")
# Normalize 
if STEREO:
    audio_l_n = normalize(audio_l)
    audio_r_n = normalize(audio_r)
        
    ## FFT Shift
    print("FFT...")
    print("Processing Left channel...")
    audio_l_n_fft = np.fft.fft(audio_l_n)
    print ("Processing Right Channel...")
    audio_r_n_fft = np.fft.fft(audio_r_n)
    print("Normalize again")
    audio_l_n_fft_n = normalize(audio_l_n_fft)
    audio_r_n_fft_n = normalize(audio_l_n_fft)
    print("Create center channel...")
    audio_c_fft = audio_l_n_fft_n + audio_r_n_fft_n
    
    # Process Vector products
    print("Computing Roots")
    roots_0 = np.zeros(len(audio_c_fft))
    roots_1 = np.zeros(len(audio_c_fft))
    for i in range(0, len(audio_l_n_fft_n)):
        c_dot_c = np.dot(audio_c_fft[i], audio_c_fft[i])
        c_dot_l_r = np.dot(audio_c_fft[i], (audio_l_n_fft_n[i] + audio_r_n_fft_n[i]))
        l_dot_r = np.dot(audio_l_n_fft_n[i], audio_r_n_fft_n[i])
        root = np.roots([abs(c_dot_c), abs(c_dot_l_r), abs(l_dot_r)])
        roots_0[i] = root [0]
        roots_1[i] = root [1]
        #print(f"{i} is {roots}")
    print ("Computing New Channels...")
    audio_out_c_fft = roots_0 * audio_c_fft
    audio_out_l_fft = audio_l_n_fft_n - audio_out_c_fft
    audio_out_r_fft = audio_r_n_fft_n - audio_out_c_fft

    print("Back to Time Domain")
    audio_out_c = np.fft.ifft(audio_out_c_fft)
    audio_out_l = np.fft.ifft(audio_out_l_fft)
    audio_out_r = np.fft.ifft(audio_out_l_fft)

    print("Normalize again...")
    audio_out_c_n = normalize(audio_out_c)
    audio_out_l_n = normalize(audio_out_r)
    audio_out_r_n = normalize(audio_out_r)
else:
    sys.exit(1)


Normalizing Numpy Arrays....
FFT...
Processing Left channel...
Processing Right Channel...
Normalize again
Create center channel...
Computing Roots
Computing New Channels...
Back to Time Domain
Normalize again...


In [10]:
print("Writing Output....")
wavfile.write(filename="voice.wav", rate=samp_rate, data=abs(audio_out_c_n))
wavfile.write(filename="left-voice-removed.wav", rate=samp_rate, data=abs(audio_out_l_n))
wavfile.write(filename="right-voice-removed.wav", rate=samp_rate, data=abs(audio_out_r_n))
ipd.Audio(abs(audio_out_c_n), rate=samp_rate)

Writing Output....
